# Working experiment

In [1]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys

In [2]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 27 # Change the range so the process is faster

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = True # speeds up the procees but showing slighly worse results
resample_frequency = 50 # in Hz, original freq is 500Hz

In [3]:
from toolbox.methods import read_data
all_epochs, all_rawdata = read_data (task = 'main', resample=resample, amount_of_subjects=amount_of_subjects, resample_frequency=resample_frequency)

Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency 

In [16]:
all_rawdata[0]['metadata'].columns

Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

# Mean acc for all subjects

In [4]:
# '''30s per subject, 13min for 26 subjects '''
# 
# import random
# from toolbox.methods import train_timepoints
# 
# subject_acc = [None]*amount_of_subjects
# timepoint_accuracy = [None]*amount_of_subjects
# 
# bool_shuffled = False
# 
# for subj in range(amount_of_subjects-1):
#     
#     print('Working on subj: ', subj)
#     X = all_rawdata[subj]['epoch_dat']
#     y = np.where(all_rawdata[subj]['metadata']['r_map'] == 45, 1, 0)
#     y2 = all_rawdata[subj]['metadata']['deci']
#     y3 = [a ^ b for a, b in zip(y, y2)]
#     if bool_shuffled: random.shuffle(y3)
#     numT = X.shape[2]
#     timepoint_accuracy[subj] = [None]*numT
#     for timepoint in range(numT):
#         # print(timepoint)
#         X_training = X[:,:,timepoint]
#         timepoint_accuracy[subj][timepoint] = train_timepoints(X_training, y3, verbose=False, acc_only=True)
# 
# numpy_timepoint_accuracy = np.array(timepoint_accuracy)

In [7]:
'''mean 158 min 50 sec execution mac m1 for every 10'''

import random
from toolbox.methods import train_timepoints

number_of_experiments = 10
bool_shuffled = True
for experiment in range(0, number_of_experiments+0):
    print('Experiment: ', experiment)
    subject_acc = [None]*amount_of_subjects
    timepoint_accuracy = [None]*amount_of_subjects
    for subj in range(amount_of_subjects-1):
        X = all_rawdata[subj]['epoch_dat']
        y = all_rawdata[subj]['metadata']['deci']
        y = np.array(y)

        # remove weights
        ymin_index = min(np.unique(y, return_counts=True)[1])
        y_index = np.where(y == 0)[0][:ymin_index]
        y_index2 = np.where(y == 1)[0][:ymin_index]
        y_index = np.concatenate((y_index, y_index2))
        X = X[y_index,:,:]
        y = y[y_index]
        # print(np.unique(y, return_counts=True))        

        if bool_shuffled: random.shuffle(y)
        
        numT = X.shape[2]
        timepoint_accuracy[subj] = [None]*numT
        for timepoint in range(numT):
            # print(timepoint)
            X_training = X[:,:,timepoint]
            timepoint_accuracy[subj][timepoint] = train_timepoints(X_training, y, verbose=False, acc_only=True)
    numpy_timepoint_accuracy = np.array(timepoint_accuracy)
    np.save('results_deci_rf/timepoint_decision_shuffled'+str(experiment)+'.npy', numpy_timepoint_accuracy)

Experiment:  0


/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_75245/2111430874.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)


Experiment:  1


/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_75245/2111430874.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)


Experiment:  2


...

/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_75245/2111430874.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)


Experiment:  8


/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_75245/2111430874.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)


Experiment:  9


/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_75245/2111430874.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)


In [6]:
numpy_timepoint_accuracy

array([list([0.6666666666666666, 0.7272727272727273, 0.6363636363636364, 0.8484848484848485, 0.8181818181818182, 0.7272727272727273, 0.7878787878787878, 0.7272727272727273, 0.696969696969697, 0.7878787878787878, 0.7878787878787878, 0.7575757575757576, 0.696969696969697, 0.7272727272727273, 0.7575757575757576, 0.8787878787878788, 0.5454545454545454, 0.7878787878787878, 0.7272727272727273, 0.7575757575757576, 0.7575757575757576, 0.7878787878787878, 0.696969696969697, 0.696969696969697, 0.7272727272727273, 0.6363636363636364, 0.8787878787878788, 0.8484848484848485, 0.696969696969697, 0.6363636363636364, 0.7272727272727273, 0.7878787878787878, 0.696969696969697, 0.6363636363636364, 0.696969696969697, 0.7575757575757576, 0.6363636363636364, 0.696969696969697, 0.7272727272727273, 0.696969696969697, 0.696969696969697, 0.7272727272727273, 0.7575757575757576, 0.7575757575757576, 0.7272727272727273, 0.7272727272727273, 0.7878787878787878, 0.7272727272727273, 0.7878787878787878, 0.787878787878787